# Recurrent neural network

- Useful for NLP due to "memory".
- Notation
    - $[l]$: $l^{th}$ layer
    - $(i)$: $i^{th}$ example
    - $\langle t \rangle$: $t^{th}$ timestamp
    - $i$: $i^{th}$ entry of a vector
- Input $x$ is represented by a 3-D tensor $(n_{x},m,T_{x})$
    - $n_{x}$: number of units (a single timestamp, a single example) For example, a language with 5000 words one-hot coded into a vector.
    - $m$: number of training example.
    - $T_{x}$: number of time steps.
- Hidden state $a$ is similarly represented by a 3-D tensor $(n_{a},m,T_{x})$
- Prediction $\hat{y}$ is similarly represented by a 3-D tensor $(n_{y},m,T_{y})$
    - $T_{y}$: number of time steps in the prediction.

## Single RNN cell

- Input
    - $x^{\langle t \rangle}$: current input.
    - $a^{\langle t-1 \rangle}$: previous hidden state.
- Output
    - $a^{\langle t \rangle}$: given to next RNN.
    - $\hat{y}^{\langle t \rangle}$: current prediction. 
- Parameters
    - The weights and biases $(W_{aa}, b_{a}, W_{ax}, b_{x})$ are re-used each time step.
    
$$a^{\langle t \rangle} = \tanh(W_{aa} a^{\langle t-1 \rangle} + W_{ax} x^{\langle t \rangle} + b_a)$$

$$\hat{y}^{\langle t \rangle} = softmax(W_{ya} a^{\langle t \rangle} + b_y)$$

In [1]:
import numpy as np

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [4]:
def rnn_cell_forward(xt, a_prev, parameters):
    """
    Implements a single forward step of RNN-cell.

    Arguments:
    xt -- Input data at timestep "t", numpy array of shape (n_x, m)
    a_prev -- Hidden state at timestep "t-1", numpy array of shape (n_a, m)
    parameters -- Python dictionary containing:
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
        ba --  Bias, numpy array of shape (n_a, 1)
        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)
        
    Returns:
    a_next -- Next hidden state, numpy array of shape (n_a, m)
    yt_pred -- Prediction at timestep "t", numpy array of shape (n_y, m)
    cache -- Tuple of values needed for the backward pass, contains (a_next, a_prev, xt, parameters)
    """
    
    # Retrieve parameters from "parameters"
    Wax = parameters["Wax"]
    Waa = parameters["Waa"]
    Wya = parameters["Wya"]
    ba = parameters["ba"]
    by = parameters["by"]
    
    a_next = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, xt) + ba)
    yt_pred = softmax(np.dot(Wya, a_next) + by)   
    
    # store values you need for backward propagation in cache
    cache = (a_next, a_prev, xt, parameters)
    
    return a_next, yt_pred, cache

## RNN forward path

In [3]:
def rnn_forward(x, a0, parameters):
    """
    Implement the forward propagation of RNN.

    Arguments:
    x -- Input data for every time-step, numpy array of shape (n_x, m, T_x).
    a0 -- Initial hidden state, numpy array of shape (n_a, m)
    parameters -- Python dictionary containing:
        Waa -- Weight matrix multiplying the hidden state, numpy array of shape (n_a, n_a)
        Wax -- Weight matrix multiplying the input, numpy array of shape (n_a, n_x)
        Wya -- Weight matrix relating the hidden-state to the output, numpy array of shape (n_y, n_a)
        ba --  Bias numpy array of shape (n_a, 1)
        by -- Bias relating the hidden-state to the output, numpy array of shape (n_y, 1)

    Returns:
    a -- Hidden states for every time-step, numpy array of shape (n_a, m, T_x)
    y_pred -- Predictions for every time-step, numpy array of shape (n_y, m, T_x)
    caches -- Tuple of values needed for the backward pass, contains (list of caches, x)
    """
    
    # Initialize "caches" which will contain the list of all caches
    caches = []
    
    # Retrieve dimensions from shapes of x and parameters["Wya"]
    n_x, m, T_x = x.shape
    n_y, n_a = parameters["Wya"].shape
        
    # initialize "a" and "y_pred" with zeros (≈2 lines)
    a = np.zeros((n_a, m, T_x))
    y_pred = np.zeros((n_y, m, T_x))
    
    # Initialize a_next (≈1 line)
    a_next = a0
    
    # loop over all time-steps
    for t in range(T_x):
        # Update next hidden state, compute the prediction, get the cache
        a_next, yt_pred, cache = rnn_cell_forward(x[:,:,t], a[:,:,t], parameters)
        # Save the value of the new "next" hidden state in a (≈1 line)
        a[:,:,t] = a_next
        # Save the value of the prediction in y
        y_pred[:,:,t] = yt_pred
        # Append "cache" to "caches"
        caches.append(cache)
    
    # store values needed for backward propagation in cache
    caches = (caches, x)
    
    return a, y_pred, caches

## RNN Limitations

- Vanishing gradient

## LSTM 

- Forget gate
    - A tensor containing values between $0$ and $1$.
        - If a unit is close to $0$, LSTM forgets the previous cell state.
        - If a unit is close to $1$, LSTM remembers the previous cell state.

$$\mathbf{\Gamma}_f^{\langle t \rangle} = \sigma(\mathbf{W}_f[\mathbf{a}^{\langle t-1 \rangle}, \mathbf{x}^{\langle t \rangle}] + \mathbf{b}_f)$$